In [1]:
# !pip install transformers accelerate
# !pip install transformers[sentencepiece]
# !pip install sentencepiece
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv
# !pip install openprompt
import os
import csv
from openprompt.data_utils import InputExample

from datasets import Dataset, load_metric
from sklearn import preprocessing
# notebook_login()
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
label2id = {'positive':0,'negative':1}

In [3]:
train = pd.read_csv(r'dataset I/train.csv', encoding='utf_8_sig')
train.fillna('', inplace=True)
train = train[['label','text']].reset_index(drop=True)

test = pd.read_csv(r'dataset I/test.csv', encoding='utf_8_sig')
test.fillna('', inplace=True)

test = test[['label','text']].reset_index(drop=True)

In [4]:
train

,label,text
0,negative,Fix macro names - related to #448
1,negative,9.0.4
2,positive,https://github.com/ImageMagick/ImageMagick/iss...
3,negative,test: incr. disk size for TEST 35 ISCSI-MULTI
4,negative,git-svn-id: https://plugins.svn.wordpress.org/...
5,negative,[JBPM-9474] Produce events to Kafka by Signal ...
6,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...
7,positive,Normalize resource URL in ResourceServlet\n\nI...
8,negative,Merge branch '1.8.x'
9,negative,Change distribution URL.\n\ngit-svn-id: https:...


In [5]:
train = train.replace({'label':label2id})
test = test.replace({'label':label2id})

In [6]:
train['label'].value_counts()

label
1    31
0    19
Name: count, dtype: int64

In [7]:
from openprompt.data_utils import InputExample

dataset = {}
# for split in ['train','test']:
    # dataset[split] = []
dataset['train'] = []
dataset['test'] = []
for index, row in train.iterrows():
# print(row.text)
# print(row['text'])
  input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
  dataset['train'].append(input_example)
for index, row in test.iterrows():
    input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
    dataset['test'].append(input_example)

# for index, row in test.iterrows():
#     input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
#     dataset['test'].append(input_example)
print(dataset['train'][0])

{
  "guid": null,
  "label": 1,
  "meta": {},
  "text_a": "Fix macro names - related to #448",
  "text_b": "",
  "tgt_text": null
}



In [8]:
len(train)

50

In [9]:
len(test)

10062

In [10]:
from openprompt.plms import load_plm

plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")


/usr/local/miniconda3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [11]:
tuned_params = filter(lambda p: p.requires_grad, plm.parameters())
num_tuned_params = sum([p.numel() for p in tuned_params])

print(f"Number of Tuned Parameters: {num_tuned_params}")

Number of Tuned Parameters: 222903552


In [12]:
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate
template_text = 'This {"placeholder":"text_a"} is a {"mask"} commit.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'This', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' Fix macro names - related to #448', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' is a', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' commit.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 1}]


In [13]:
# WrapperClass.save_pretrained(nb_fname)

In [14]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))


{'input_ids': [100, 14269, 11663, 3056, 3, 18, 1341, 12, 24156, 3707, 19, 3, 9, 32099, 10042, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [0, 32099, 0], 'loss_ids': [0, 1, 0]}
['▁This', '▁Fix', '▁macro', '▁names', '▁', '-', '▁related', '▁to', '▁#4', '48', '▁is', '▁', 'a', '<extr

In [15]:
model_inputs = {}
for split in ['train', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors


In [16]:
# We provide a `PromptDataLoader` class to help you do all the above matters and wrap them into an `torch.DataLoader` style iterator.
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
# next(iter(train_dataloader))

tokenizing: 50it [00:00, 557.61it/s]


In [17]:
from openprompt.prompts import ManualVerbalizer,KnowledgeableVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
# myverbalizer = ManualVerbalizer(tokenizer, num_classes=3,
#                         label_words=[["Adaptive"], ["Perfective"], ["Corrective"]])
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=2).from_file("knowledgeable_verbalizer-I.txt")

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


##Num of label words for each label: [21, 20]
Parameter containing:
tensor([[[ 2451,     0,     0,     0,     0],
         [ 3614,     0,     0,     0,     0],
         [ 1346,     0,     0,     0,     0],
         [  766,     0,     0,     0,     0],
         [ 7992,     0,     0,     0,     0],
         [ 1006,    35,     0,     0,     0],
         [    3, 11005,   524,     0,     0],
         [   16,  4334,     0,     0,     0],
         [    3, 21477,     0,     0,     0],
         [   36,  5595,     0,     0,     0],
         [   16, 12388,   687,   179,     0],
         [    3, 23094,     0,     0,     0],
         [  417,     0,     0,     0,     0],
         [ 3442,     0,     0,     0,     0],
         [  143,     0,     0,     0,     0],
         [ 2210,     0,     0,     0,     0],
         [ 1669,     0,     0,     0,     0],
         [11892,    15,     0,     0,     0],
         [   21,    17,  4921,     0,     0],
         [ 4095,     0,     0,     0,     0],
         [11

In [18]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


In [19]:
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)



/usr/local/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
import ipynbname
nb_fname = ipynbname.name()
print(nb_fname)

K-Base-PKG_I


In [21]:
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 10062it [00:18, 539.21it/s]


In [22]:
# Evaluate
# Set up early stopping parameters
best_validation_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
waited_epochs = 0

for epoch in tqdm(range(10)):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step % 100 == 1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss / (step + 1)), flush=True)

    # Validation loop
    validation_loss = 0
    prompt_model.eval()
    with torch.no_grad():
        for val_step, val_inputs in enumerate(validation_dataloader):
            if use_cuda:
                val_inputs = val_inputs.cuda()
            val_logits = prompt_model(val_inputs)
            val_labels = val_inputs['label']
            val_loss = loss_func(val_logits, val_labels)
            validation_loss += val_loss.item()

    validation_loss /= (val_step + 1)
    print("Epoch {}, validation loss: {}".format(epoch, validation_loss), flush=True)

    # Check for early stopping
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        waited_epochs = 0
        # Save the model's state dictionary to a file
        # torch.save(prompt_model.state_dict(), nb_fname +'.pth')
    else:
        waited_epochs += 1
        if waited_epochs >= patience:
            print("Early stopping at epoch {}.".format(epoch), flush=True)
            break

    prompt_model.train()  # Set the model back to training mode

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0, average loss: 0.9051495492458344
Epoch 0, validation loss: 0.6283000691161625


 10%|█         | 1/10 [01:57<17:35, 117.32s/it]

Epoch 1, average loss: 0.5262957811355591
Epoch 1, validation loss: 0.6150568198406734


 20%|██        | 2/10 [03:54<15:37, 117.19s/it]

...

 30%|███       | 3/10 [05:51<13:39, 117.03s/it]

Epoch 3, average loss: 0.2977285087108612
Epoch 3, validation loss: 0.7615742687889581


 40%|████      | 4/10 [07:48<11:41, 116.99s/it]

Epoch 4, average loss: 0.2764476388692856
Epoch 4, validation loss: 0.7688264773421238
Early stopping at epoch 4.


 40%|████      | 4/10 [09:45<14:37, 146.28s/it]


In [23]:
from sklearn.metrics import classification_report
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

# Generate the classification report
# 这里是数字所以要转字符
class_names = list(test['label'].unique())  # Replace num_classes with the number of classes in your dataset
classification_rep = classification_report(alllabels, allpreds, digits=4)

print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0     0.5593    0.5400    0.5495      3746
           1     0.7327    0.7476    0.7401      6316

    accuracy                         0.6703     10062
   macro avg     0.6460    0.6438    0.6448     10062
weighted avg     0.6681    0.6703    0.6691     10062



In [24]:
# # store the checkpoints
# prompt_model.plm.save_pretrained(nb_fname)
# model_config.save_pretrained(nb_fname)
# myverbalizer.tokenizer.save_pretrained(nb_fname)

In [25]:
# import os
# os.system('shutdown')

In [26]:
# train.to_csv('dataset II/train.csv',encoding='utf_8_sig')
# test.to_csv('dataset II/test.csv', encoding='utf_8_sig')

In [27]:
tuned_params = filter(lambda p: p.requires_grad, prompt_model.parameters())
num_tuned_params = sum([p.numel() for p in tuned_params])

print(f"Number of Tuned Parameters: {num_tuned_params}")

Number of Tuned Parameters: 222903594
